In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
X.shape

In [14]:
%run ../src/models/simple_feed_forward.py

[42144.371]
Iter=10, Batch Loss=[42144.371]
[33636.211]
Iter=20, Batch Loss=[33636.211]
[31504.031]
Iter=30, Batch Loss=[31504.031]
[40318.102]
Iter=40, Batch Loss=[40318.102]
[32625.121]
Iter=50, Batch Loss=[32625.121]
[39429.418]
Iter=60, Batch Loss=[39429.418]
[33164.312]
Iter=70, Batch Loss=[33164.312]
[29551.734]
Iter=80, Batch Loss=[29551.734]
[36044.0]
Iter=90, Batch Loss=[36044.0]
[26750.223]
Iter=100, Batch Loss=[26750.223]
[30645.049]
Iter=110, Batch Loss=[30645.049]
[24948.734]
Iter=120, Batch Loss=[24948.734]
[25974.473]
Iter=130, Batch Loss=[25974.473]
[29437.652]
Iter=140, Batch Loss=[29437.652]
[30375.186]
Iter=150, Batch Loss=[30375.186]
[28367.152]
Iter=160, Batch Loss=[28367.152]
[28349.021]
Iter=170, Batch Loss=[28349.021]
[31491.424]
Iter=180, Batch Loss=[31491.424]
[24604.65]
Iter=190, Batch Loss=[24604.65]
[25915.705]
Iter=200, Batch Loss=[25915.705]
[28497.473]
Iter=210, Batch Loss=[28497.473]
[28631.049]
Iter=220, Batch Loss=[28631.049]
[28239.051]
Iter=230, Bat

KeyboardInterrupt: 

In [ ]:
%run ../src/models/feature_loader.py
feature_loader = FeatureTensorLoader(lags=lags, check_integrity=False, batch_size=2)
features, targets = feature_loader.load_batch(2)

In [ ]:
learning_rate = 0.001
max_iter = 200000
batch_size = 128
display_step = 10
n = m = 300
lags = 2

feature_loader = FeatureTensorLoader(lags=lags, check_integrity=False)

def init_weights(shape):
    weights = tf.random_normal(shape, stddev=0.1)
    return tf.Variable(weights)

X = tf.placeholder(tf.float32, [lags, n, m])
Y = tf.placeholder(tf.float32, [n, m])

# Store layers weight & bias
weights = {
    'w1': tf.Variable(
        tf.random_normal([lags, n, m])),
}

biases = {
    'b1': tf.Variable(tf.random_normal([n, m])),
}

def create_network(X, weights, biases):
    l1 = tf.nn.relu(X)
    l1 = tf.add(tf.matmul(l1, weights['w1']), biases['b1'])
    l1 = tf.nn.sigmoid(l1)
    return l1

network = create_network(X, weights, biases)
square_errors = tf.nn.l2_loss(t=network)
cost = tf.reduce_mean(square_errors)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    batch = 1
    while batch * batch_size < max_iter:
        batch_features, batch_targest = feature_loader(batch)
        sess.run(optimizer, feed_dict={x: batch_features,
                                       y: batch_targets})
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss = sess.run([cost], feed_dict={x: batch_features,
                                               y: batch_targets})
            print("Iter " + str(step*batch_size) + ", Batch Loss= " + \
                  "{:.2f}".format(loss))
        batch += 1
    print("Optimization Finished!")
                  


In [ ]:

    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                      y: mnist.test.labels[:256],
                                      keep_prob: 1.}))

In [ ]:
# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                       keep_prob: dropout})
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              keep_prob: 1.})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for 256 mnist test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                      y: mnist.test.labels[:256],
                                      keep_prob: 1.}))

In [ ]:
# Implementation of a simple MLP network with one hidden layer. Tested on the iris data set.
# Requires: numpy, sklearn>=0.18.1, tensorflow>=1.0

# NOTE: In order to make the code simple, we rewrite x * W_1 + b_1 = x' * W_1'
# where x' = [x | 1] and W_1' is the matrix W_1 appended with a new row with elements b_1's.
# Similarly, for h * W_2 + b_2
import tensorflow as tf
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42
tf.set_random_seed(RANDOM_SEED)


def init_weights(shape):
    """ Weight initialization """
    weights = tf.random_normal(shape, stddev=0.1)
    return tf.Variable(weights)

def forwardprop(X, w_1, w_2):
    """
    Forward-propagation.
    IMPORTANT: yhat is not softmax since TensorFlow's softmax_cross_entropy_with_logits() does that internally.
    """
    h    = tf.nn.sigmoid(tf.matmul(X, w_1))  # The \sigma function
    yhat = tf.matmul(h, w_2)  # The \varphi function
    return yhat

def get_iris_data():
    """ Read the iris data set and split them into training and test sets """
    iris   = datasets.load_iris()
    data   = iris["data"]
    target = iris["target"]

    # Prepend the column of 1s for bias
    N, M  = data.shape
    all_X = np.ones((N, M + 1))
    all_X[:, 1:] = data

    # Convert into one-hot vectors
    num_labels = len(np.unique(target))
    all_Y = np.eye(num_labels)[target]  # One liner trick!
    return train_test_split(all_X, all_Y, test_size=0.33, random_state=RANDOM_SEED)

def main():
    train_X, test_X, train_y, test_y = get_iris_data()

    # Layer's sizes
    x_size = train_X.shape[1]   # Number of input nodes: 4 features and 1 bias
    h_size = 256                # Number of hidden nodes
    y_size = train_y.shape[1]   # Number of outcomes (3 iris flowers)

    # Symbols
    X = tf.placeholder("float", shape=[None, x_size])
    y = tf.placeholder("float", shape=[None, y_size])

    # Weight initializations
    w_1 = init_weights((x_size, h_size))
    w_2 = init_weights((h_size, y_size))

    # Forward propagation
    yhat    = forwardprop(X, w_1, w_2)
    predict = tf.argmax(yhat, axis=1)

    # Backward propagation
    cost    = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=yhat))
    updates = tf.train.GradientDescentOptimizer(0.01).minimize(cost)

    # Run SGD
    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)

    for epoch in range(100):
        # Train with each example
        for i in range(len(train_X)):
            sess.run(updates, feed_dict={X: train_X[i: i + 1], y: train_y[i: i + 1]})

        train_accuracy = np.mean(np.argmax(train_y, axis=1) ==
                                 sess.run(predict, feed_dict={X: train_X, y: train_y}))
        test_accuracy  = np.mean(np.argmax(test_y, axis=1) ==
                                 sess.run(predict, feed_dict={X: test_X, y: test_y}))

        print("Epoch = %d, train accuracy = %.2f%%, test accuracy = %.2f%%"
              % (epoch + 1, 100. * train_accuracy, 100. * test_accuracy))

    sess.close()

if __name__ == '__main__':
    main()